<a href="https://colab.research.google.com/github/pierredumontel/plane-classification/blob/main/notebooks/train_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!curl -O https://www.robots.ox.ac.uk/~vgg/data/fgvc-aircraft/archives/fgvc-aircraft-2013b.tar.gz
!tar zxf fgvc-aircraft-2013b.tar.gz
!mv fgvc-aircraft-2013b dataset 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2625M  100 2625M    0     0  28.7M      0  0:01:31  0:01:31 --:--:-- 29.2M


In [8]:
import pathlib

import pandas as pd 

In [9]:
pd.__version__

'1.3.5'

In [10]:
DATA_DIR = pathlib.Path('dataset/data')

### Make assumptions about number of spaces (here=2)

In [20]:
manufacturer_df = pd.read_csv(DATA_DIR / 'images_manufacturer_train.txt',sep=' ',
            names=['image_id','m1','m2'],
            usecols=['image_id','m1','m2'],#usecols for v1.4 compatibility
            dtype={'image_id':str}) #ids are not int but string 
manufacturer_df['manufacturer']=manufacturer_df['m1']+' '+manufacturer_df['m2']
manufacturer_df['manufacturer'].unique()

array([nan, 'British Aerospace', 'Lockheed Corporation',
       'Douglas Aircraft', 'McDonnell Douglas', 'de Havilland',
       'Lockheed Martin', 'Dassault Aviation', 'Bombardier Aerospace',
       'Gulfstream Aerospace', 'Cirrus Aircraft'], dtype=object)

In [28]:
! grep ',' dataset/data/images_manufacturer_train.txt

In [31]:
#Recherche le caractère T dans le fichier et n'affiche que trois lignes (head -3)
! grep 'T' dataset/data/images_manufacturer_train.txt | head -3

0724121 ATR
0619697 ATR
2243949 ATR


In [37]:
#wc : Compte le nombre d'élements (-l : ligne, -c : caractère, -w : word)
! grep 'T' dataset/data/images_manufacturer_train.txt | wc 

    132     264    1848


In [38]:
!cut -f 1 -d' ' dataset/data/images_manufacturer_train.txt | head

1025794
1340192
0056978
0698580
0450014
1042824
0894380
1427680
0817494
0716386


In [25]:
manufacturer_df = pd.read_csv(DATA_DIR / 'images_manufacturer_train.txt',sep='\t',
            names=['all'],
            dtype={'all':str}) #ids are not int but string 


#La fonction split() découpe sur une chaîne de caractères 
manufacturer_df['image_id'] = manufacturer_df['all'].apply(lambda x:x.split(' ')[0])  

#La fonction '<car>'.join(liste) concatène les éléments de liste en utillisant le séparateur <car>
manufacturer_df['manufacturer']=manufacturer_df['all'].apply(lambda x:' '.join(x.split(' ')[1:])) 
manufacturer_df['manufacturer'].unique()

array(['Boeing', 'Airbus', 'ATR', 'Antonov', 'British Aerospace',
       'Beechcraft', 'Lockheed Corporation', 'Douglas Aircraft Company',
       'Canadair', 'Cessna', 'McDonnell Douglas', 'de Havilland', 'Robin',
       'Dornier', 'Embraer', 'Eurofighter', 'Lockheed Martin',
       'Dassault Aviation', 'Fokker', 'Bombardier Aerospace',
       'Gulfstream Aerospace', 'Ilyushin', 'Fairchild', 'Piper',
       'Cirrus Aircraft', 'Saab', 'Supermarine', 'Panavia', 'Tupolev',
       'Yakovlev'], dtype=object)

Verify data 

In [14]:
manufacturer_df['manufacturer'].value_counts(dropna=False)

Boeing         733
Airbus         434
Embraer        233
McDonnell      232
de             167
Canadair       134
Douglas        133
Cessna         133
British        133
Lockheed       102
Fokker         100
Dassault        67
Gulfstream      67
Beechcraft      67
Saab            67
Tupolev         66
ATR             66
Panavia         34
Antonov         34
Dornier         34
Yakovlev        34
Bombardier      33
Ilyushin        33
Fairchild       33
Piper           33
Cirrus          33
Eurofighter     33
Supermarine     33
Robin           33
Name: manufacturer, dtype: int64

Verify missing data

In [15]:
manufacturer_df.isna().sum()

image_id        0
manufacturer    0
dtype: int64

In [18]:
assert manufacturer_df['image_id'].isna().sum()==0,"Missing value in image_id"
assert manufacturer_df['manufacturer'].isna().sum()==0,"Missing value in manufacturer"